In [2]:
from splink.duckdb.duckdb_linker import DuckDBLinker

In [3]:
import pandas as pd 
pd.options.display.max_rows = 1000
df = pd.read_parquet("./data/historical_figures_with_errors_50k.parquet")
df.head(5)

,uncorrupted_record,cluster,full_name,dob,birth_place,postcode_fake,lat,lng,gender,occupation,unique_id
0,True,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",1630-08-01,Devon,TQ13 8DF,50.692449,-3.813964,male,politician,Q2296770-1
1,False,Q2296770,thomas of chudleigh,1630-08-01,Devon,TQ13 8DF,50.692449,-3.813964,male,politician,Q2296770-2
2,False,Q2296770,tom 1st baron clifford of chudleigh,1630-08-01,Devon,TQ13 8DF,50.692449,-3.813964,male,politician,Q2296770-3
3,False,Q2296770,thomas 1st chudleigh,1630-08-01,Devon,TQ13 8HU,50.687638,-3.895877,None,politician,Q2296770-4
4,False,Q2296770,"thomas clifford, 1st baron chudleigh",1630-08-01,Devon,TQ13 8DF,50.692449,-3.813964,None,politician,Q2296770-5


In [4]:
import numpy as np
import pandas as pd

def clean_df(df):
    cols = [
        "unique_id",
        "cluster",
        "full_name",
        "dob",
        "birth_place",
        "postcode_fake",
        "gender",
        "occupation",
    ]

    df = df[cols].copy()

    df["name_split"] = df["full_name"].str.strip().str.split(" ")
    df["name_split_length"] = df["name_split"].str.len()
    df["first_name"] = df["name_split"].str[0]
    df["surname"] = df["name_split"].str[-1]
    df["surname"] = np.where(df["name_split_length"] > 1, df["surname"], "")
    # df["middle_names"] = df["name_split"].str[1:-1]

    df["first_and_surname"] = df["first_name"] + " " + df["surname"]

    for col in [
        "full_name",
        "first_and_surname",
        "first_name",
        "surname",
        "dob",
        "birth_place",
        "postcode_fake",
        "gender",
        "occupation",
    ]:
        df[col] = df[col].str.lower().str.strip()
        df[col] = df[col].replace({"": None})

    cols = [
        "unique_id",
        "cluster",
        "full_name",
        "first_and_surname",
        "first_name",
        "surname",
        "dob",
        "birth_place",
        "postcode_fake",
        "gender",
        "occupation",
    ]
    return df[cols]


df_clean = clean_df(df)
df_clean.head(2)

,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician


In [5]:
# Initialise the linker, passing in the input dataset(s)
linker = DuckDBLinker(input_tables = {"df": df_clean})

linker.profile_columns(["first_name", "postcode_fake", "substr(dob, 1,4)"], top_n=10, bottom_n=5)

In [12]:

from splink.comparison_library import exact_match, levenshtein
settings = {
    "proportion_of_matches": 1e-5,
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.surname = r.surname",
        "l.surname = r.surname and l.dob = r.dob",
        "l.first_name = r.first_name and l.dob = r.dob",
        "l.postcode_fake = r.postcode_fake and l.first_name = r.first_name",
    ],
    "comparisons": [
        levenshtein("first_name", 2, term_frequency_adjustments=True),
        levenshtein("surname", 2, term_frequency_adjustments=True),
        levenshtein("dob", 2, term_frequency_adjustments=True),
        levenshtein("postcode_fake", 2),
        exact_match("birth_place", term_frequency_adjustments=True),
        exact_match("occupation",  term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,

    "max_iterations": 10,
    "em_convergence": 0.01
}

In [13]:
linker.initialise_settings(settings)
linker.train_u_using_random_sampling(target_rows=4e6)

In [14]:
blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_names = linker.train_m_using_expectation_maximisation(blocking_rule)
# training_session_names.match_weights_interactive_history_chart()

Iteration 0: Largest change in params was 0.16 in the m_probability of dob, level `Levenstein <= 2`
Iteration 1: Largest change in params was -0.013 in the m_probability of dob, level `All other comparisons`
Iteration 2: Largest change in params was -0.00372 in the m_probability of dob, level `All other comparisons`
EM converged after 2 iterations
Proportion of matches not fully trained, current estimates are [9.878092853498286e-06, 0.00019493379326168973, 9.878092853498286e-06]


In [15]:
blocking_rule = "l.dob = r.dob"
training_session_dob = linker.train_m_using_expectation_maximisation(blocking_rule)

Iteration 0: Largest change in params was 0.139 in the m_probability of first_name, level `All other comparisons`
Iteration 1: Largest change in params was 0.0417 in the m_probability of first_name, level `All other comparisons`
Iteration 2: Largest change in params was 0.00604 in the m_probability of first_name, level `All other comparisons`
EM converged after 2 iterations
Proportion of matches can now be estimated, estimates are [1.119776837233541e-05, 0.00019493379326168973, 1.119776837233541e-05, 0.00019493379326168973]


The final match weights can be viewed in the match weights chart:

In [16]:
linker.settings_obj.match_weights_chart()

In [17]:
df_e = linker.predict().as_pandas_dataframe()

print(len(df_e))

307023


In [18]:
df_e.head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,bf_tf_adj_first_name,surname_l,surname_r,gamma_surname,bf_surname,bf_tf_adj_surname,dob_l,dob_r,gamma_dob,bf_dob,bf_tf_adj_dob,postcode_fake_l,postcode_fake_r,gamma_postcode_fake,bf_postcode_fake,birth_place_l,birth_place_r,gamma_birth_place,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,bf_occupation,bf_tf_adj_occupation,match_key
0,7.583356,0.994813,Q2296770-1,Q2296770-9,thomas,thomas,2,41.302961,0.46687,chudleigh,chudleigh,2,1204.109432,2.136662,1630-08-01,1630-05-01,1,3.707191,1.0,tq13 8df,NaN,-1,1.0,devon,NaN,-1,1.0,1.0,politician,politician,1,21.387992,0.473004,0
1,7.583356,0.994813,Q2296770-2,Q2296770-9,thomas,thomas,2,41.302961,0.46687,chudleigh,chudleigh,2,1204.109432,2.136662,1630-08-01,1630-05-01,1,3.707191,1.0,tq13 8df,NaN,-1,1.0,devon,NaN,-1,1.0,1.0,politician,politician,1,21.387992,0.473004,0
2,7.583356,0.994813,Q2296770-4,Q2296770-9,thomas,thomas,2,41.302961,0.46687,chudleigh,chudleigh,2,1204.109432,2.136662,1630-08-01,1630-05-01,1,3.707191,1.0,tq13 8hu,NaN,-1,1.0,devon,NaN,-1,1.0,1.0,politician,politician,1,21.387992,0.473004,0
3,7.583356,0.994813,Q2296770-5,Q2296770-9,thomas,thomas,2,41.302961,0.46687,chudleigh,chudleigh,2,1204.109432,2.136662,1630-08-01,1630-05-01,1,3.707191,1.0,tq13 8df,NaN,-1,1.0,devon,NaN,-1,1.0,1.0,politician,politician,1,21.387992,0.473004,0
4,7.583356,0.994813,Q2296770-6,Q2296770-9,thomas,thomas,2,41.302961,0.46687,chudleigh,chudleigh,2,1204.109432,2.136662,1630-08-01,1630-05-01,1,3.707191,1.0,tq13 8df,NaN,-1,1.0,west devon,NaN,-1,1.0,1.0,politician,politician,1,21.387992,0.473004,0


You can also view rows in this dataset as a waterfall chart as follows:

In [21]:
from splink.charts import waterfall_chart
records_to_plot = df_e.head(50).to_dict(orient="records")
waterfall_chart(records_to_plot, linker.settings_obj, filter_nulls=False)